In [ ]:
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch

# Code samples taken from: https://github.com/informationcake/SDSS-ML/blob/master/SDSS_ML.py
# Cite dataset as: A. O. Clarke, A. M. M. Scaife, R. Greenhalgh, & V. Griguta. (2020). Identifying galaxies, quasars and stars with machine learning: a new catalogue of classifications for 111 million SDSS sources without spectra [Data set]. Zenodo. [https://doi.org/10.5281/zenodo.3768398](https://doi.org/10.5281/zenodo.3768398)

In [ ]:
#Loading/saving python data objects
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

# Take input data, prepare for supervised ML or TSNE
def prepare_data(data, feature_columns, train_percent=0.5, ttsplit=True, mag_split=False, mag_lim=18.5, ttbelow=True, scale=False, verbose=False, newsources=False):
    print('Preparing data... Its shape is: {0}'.format(data.shape))
    #test/train split using sklearn function
    if ttsplit==True and mag_split==False:
        all_features = data[[*feature_columns]]
        if newsources==False:
            all_classes = data['class']
        if newsources==True:
            all_classes = data['class_pred']
        if scale==True:
            print('Scaling features...')
            #all_features = preprocessing.scale(all_features)
            all_features = preprocessing.normalize(all_features)
            all_features = pd.DataFrame(all_features)
        features_train, features_test, classes_train, classes_test = train_test_split(all_features, all_classes, train_size=train_percent, random_state=0, stratify=all_classes)
        class_names = np.unique(all_classes)
        feature_names = list(all_features)
        #print(isinstance(classes_train, pd.DataFrame))
        if verbose==True: print('feature names are: ', str(feature_names))
        return {'features_train':features_train, 'features_test':features_test, 'classes_train':classes_train, 'classes_test':classes_test, 'class_names':class_names, 'feature_names':feature_names} #return dictionary. data within dictionary are DataFrames.

    #test/train split only below a PSF magnitude limit in r band
    if ttsplit==True and mag_split==True:
        if ttbelow==True:
            print('Imposing magnitude cut of cmod_r={0}, performing test/train split below this...'.format(mag_lim))
            data_tt = data[data.psf_r < mag_lim] # split out brighter fraction of data before tt split
            data_new = data[data.psf_r > mag_lim] # set aside fraction of fainter new sources (e.g. simulating deeper data)
        if ttbelow==False:
            print('Imposing magnitude cut of cmod_r={0}, performing test/train split above this...'.format(mag_lim))
            data_tt = data[data.psf_r > mag_lim] # split out fainter fraction of data before tt split
            data_new = data[data.psf_r < mag_lim] # set aside fraction of brighter sources
        all_features = data_tt[[*feature_columns]]
        all_classes = data_tt['class']
        print('Number of sources not in tt-split: {0}'.format(len(data_new)))
        # do tt split
        features_train, features_test, classes_train, classes_test = train_test_split(all_features, all_classes, train_size=train_percent, random_state=0, stratify=all_classes)
        # append the data_new deeper sources not included in tt split to the arrays
        features_test = features_test.append( data_new[[*feature_columns]] )
        classes_test = classes_test.append( data_new['class'] )
        print('Training on \n{0}'.format(classes_train.value_counts()))
        print('Testing on \n{0}'.format(classes_test.value_counts()))
        # get names as strings
        class_names = np.unique(data['class'])
        feature_names = list(data[[*feature_columns]])
        if verbose==True: print('feature names are: ', str(feature_names))

        return {'features_train':features_train, 'features_test':features_test, 'classes_train':classes_train, 'classes_test':classes_test, 'class_names':class_names, 'feature_names':feature_names} #return dictionary


    #no test/train split, just return data in format for e.g. tsne or clustering
    if ttsplit==False:
        all_features = data[[*feature_columns]]
        all_classes = data['class']
        class_names = np.unique(data['class'])
        return {'all_features':all_features, 'all_classes':all_classes, 'class_names':class_names} #return dictionary

In [ ]:
# define feature columns used
# psf magnitudes
psf = ['psf_u', 'psf_g', 'psf_r', 'psf_i', 'psf_z']
# WISE magnitudes
wise = ['w1' ,'w2', 'w3', 'w4']
# Select columns to be used as features (typical combinations tested, commented in/out)
feature_columns = psf + wise + ['resolvedr']

df = load_obj('data/Identifying_galaxies_quasars_stars/df_spec_classprobs')
print('features used are:')
print(df[feature_columns].columns)

features used are:
Index(['psf_u', 'psf_g', 'psf_r', 'psf_i', 'psf_z', 'w1', 'w2', 'w3', 'w4',
       'resolvedr'],
      dtype='object')


In [ ]:
df.shape

(3099457, 64)

In [ ]:
print(df['class'].value_counts())

GALAXY    2209333
STAR       512236
QSO        377888
Name: class, dtype: int64


In [ ]:
data_prep_dict_all = prepare_data(df, feature_columns, ttsplit=True, mag_split=False, mag_lim=18, ttbelow=True)

Preparing data... Its shape is: (3099457, 64)


In [ ]:
# write out subsample of data for unsupervised processing:
tsne_subsample = data_prep_dict_all['features_test'].sample(10000, random_state = 42 ).values
tsne_subsample_classes = data_prep_dict_all['classes_test'].sample(10000, random_state = 42 ).values
# save out for using in unsupervised learning:
pickle.dump( (tsne_subsample,tsne_subsample_classes), open( "data/Identifying_galaxies_quasars_stars/subdf_unsupervised.pickle", "wb" ))